In [2]:
import newspaper

In [9]:
nyt_paper = newspaper.build('https://www.nytimes.com')

for article in nyt_paper.articles:
    print(article.url)

IndexError: list index out of range

In [6]:
cnn_paper = newspaper.build('http://cnn.com')

for article in cnn_paper.articles:
    print(article.url)

http://cnn.com/business/media
http://cnn.com/specials/sport/rugby-world-cup-japan-2019-spt-intl
https://www.cnn.com/business/media
https://www.cnn.com/specials/sport/rugby-world-cup-japan-2019-spt-intl
http://cnn.com/TRANSCRIPTS/2019.12.19.html
http://cnn.com/TRANSCRIPTS/2019.12.20.html
http://cnn.com/TRANSCRIPTS/2019.12.21.html
http://cnn.com/TRANSCRIPTS/2019.12.22.html
http://cnn.com/TRANSCRIPTS/2019.12.23.html
http://cnn.com/TRANSCRIPTS/2019.12.24.html
http://cnn.com/TRANSCRIPTS/2019.12.25.html
http://cnn.com/TRANSCRIPTS/2019.12.26.html
http://cnn.com/TRANSCRIPTS/2019.12.27.html
http://cnn.com/TRANSCRIPTS/2019.12.28.html
http://www.cnn.com/travel/article/top-cambodia-foods-intl-hnk/index.html
https://www.cnn.com/travel/article/fukushima-japan-tourism/index.html
https://www.cnn.com/travel/article/fun-things-desert/index.html
https://www.cnn.com/travel/article/fully-vegan-hotels-uk/index.html
https://www.cnn.com/travel/article/long-life-lerik-azerbaijan-wellness/index.html
http://cnn.

## Get links in the past month (Dec)

In [29]:
import re
import time

import requests
from bs4 import BeautifulSoup

links = []

# for December:
for i in range(1,28):
    date_str = ''
    if i < 10:
        date_str = '0'+str(i)
    else:
        date_str = str(i)
    url = 'https://www.nytimes.com/issue/todayspaper/2019/12/'+date_str+'/todays-new-york-times'
    header = {'User-Agent' : 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'}#，'Referer' : 'http://www.chinadaily.com.cn/'}
    response = requests.get(url, header)
    
    soup = BeautifulSoup(response.text,'html.parser')
    #print(soup.prettify())

    for link in soup.find_all('a', href=re.compile('2019')):
        links.append(link.get('href'))

# for today:
url = 'https://www.nytimes.com/section/todayspaper'
header = {'User-Agent' : 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'}#，'Referer' : 'http://www.chinadaily.com.cn/'}
response = requests.get(url, header)

soup = BeautifulSoup(response.text,'html.parser')
#print(soup.prettify())

for link in soup.find_all('a', href=re.compile('2019')):
    links.append(link.get('href'))

print(len(links))
links = list(dict.fromkeys(links))  # eliminate duplicate links and maintain the order
print(len(links))

1515
1109


In [32]:
links[901]

'/2019/12/22/us/politics/Michael-Bloomberg-nyc-mayor-election.html'

In [33]:
# add prefix to each link
full_links = ['https://www.nytimes.com'+i for i in links]
full_links[:10]

['https://www.nytimes.com/2019/11/30/us/politics/trump-seals-eddie-gallagher.html',
 'https://www.nytimes.com/2019/11/30/business/amazon-baltimore.html',
 'https://www.nytimes.com/2019/11/30/world/americas/venezuela-students-hunger.html',
 'https://www.nytimes.com/2019/11/30/us/politics/2020-democratic-candidates.html',
 'https://www.nytimes.com/2019/11/30/arts/latin-dictionary.html',
 'https://www.nytimes.com/2019/11/30/world/asia/philippines-mariners-cargo-ships.html',
 'https://www.nytimes.com/2019/12/01/world/asia/china-fentanyl-crackdown.html',
 'https://www.nytimes.com/2019/11/30/world/middleeast/adel-abdul-mahdi-resigns-iraq.html',
 'https://www.nytimes.com/2019/11/30/world/europe/london-bridge-attack.html',
 'https://www.nytimes.com/2019/11/30/world/europe/labour-tories-johnson-corbyn-westminster.html']

In [34]:
# output the links to store:
with open('nyt_links_12.txt', 'w') as file:
    file.write('\n'.join(full_links))

In [65]:
import pandas as pd

nyt = pd.DataFrame(columns=['title', 'timestamp', 'content', 'media', 'link'])
nyt

,title,timestamp,content,media,link


In [68]:
import time

problems = []  # collect the index that have problems
header = {'User-Agent' : 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'}#，'Referer' : 'http://www.chinadaily.com.cn/'}

for i in range(len(full_links)):
    if i == 299:
        print('crawled 300 articles ...')
    elif i == 599:
        print('crawled 600 articles ...')
    
    # sleep:
    if i != 0 and i%90 == 0:
        print('reach 90 articles. sleep 10s ...')
        time.sleep(10)
    
    entry = []
    
    response = requests.get(full_links[i], header)
    soup = BeautifulSoup(response.text,'html.parser')
    
    #title
    try:
        title = soup.find('h1').get_text().strip()
        # print(title)
        if title == '' or title is None:
            print("title for "+str(i)+" not found")
            problems.append(i)
            entry.append(None)
        else:
            entry.append(title)
    except:
        print("title for "+str(i)+" not found")
        problems.append(i)
        entry.append(None)

    # timestamp
    try:
        timestamp = soup.find('time')['datetime']
        # print(timestamp)
        if timestamp == '' or timestamp is None:
            entry.append(None)
            print("timestamp for "+str(i)+" not found")
            problems.append(i)
        else:
            entry.append(timestamp.strip())
    except:
        entry.append(None)
        print("timestamp for "+str(i)+" not found")
        problems.append(i)
        
    # content
    try:
        content = soup.find('body').find('section', {'name':'articleBody'}).get_text().strip()
        #print(content)
        if content == '' or content is None:
            entry.append(None)
            print("content for "+str(i)+" not found")
            problems.append(i)
        else:  # content is not None:
            entry.append(content)
    except:
        print("content for "+str(i)+" not found")
        entry.append(None)
        problems.append(i)


    # media
    entry.append('New York Times')

    # link
    entry.append(full_links[i])

    #print(entry)

    nyt.loc[i] = entry
    
    #time.sleep(1)

display(nyt)

reach 90 articles. sleep 10s ...
content for 97 not found
reach 90 articles. sleep 10s ...
timestamp for 262 not found
content for 262 not found
reach 90 articles. sleep 10s ...
content for 286 not found
timestamp for 287 not found
content for 287 not found
timestamp for 294 not found
crawled 300 articles ...
reach 90 articles. sleep 10s ...
content for 420 not found
reach 90 articles. sleep 10s ...
content for 497 not found
reach 90 articles. sleep 10s ...
timestamp for 573 not found
content for 573 not found
crawled 600 articles ...
timestamp for 620 not found
reach 90 articles. sleep 10s ...
content for 664 not found
content for 695 not found
reach 90 articles. sleep 10s ...
content for 742 not found
timestamp for 770 not found
reach 90 articles. sleep 10s ...
timestamp for 810 not found
timestamp for 853 not found
content for 853 not found
content for 897 not found
reach 90 articles. sleep 10s ...
content for 935 not found
reach 90 articles. sleep 10s ...
content for 1006 not found

,title,timestamp,content,media,link
0,Trump’s Intervention in SEALs Case Tests Penta...,2019-11-30T20:47:50-05:00,"He was limp and dusty from an explosion, consc...",New York Times,https://www.nytimes.com/2019/11/30/us/politics...
1,Prime Mover: How Amazon Wove Itself Into the L...,2019-11-30T06:00:18-05:00,Amazon EverywherePrime Mover: How Amazon Wove ...,New York Times,https://www.nytimes.com/2019/11/30/business/am...
2,Students Fainting From Hunger in Venezuela’s F...,2019-11-30T05:00:23-05:00,"BOCA DE UCHIRE, Venezuela — Hundreds of childr...",New York Times,https://www.nytimes.com/2019/11/30/world/ameri...
3,2020 Democratic Candidates Wage Escalating Fig...,2019-11-30T05:00:15-05:00,"WALPOLE, N.H. — Pete Buttigieg has a nifty pol...",New York Times,https://www.nytimes.com/2019/11/30/us/politics...
4,Latin Dictionary’s Journey: A to Zythum in 125...,2019-11-30T12:34:51-05:00,MUNICH — When German researchers began working...,New York Times,https://www.nytimes.com/2019/11/30/arts/latin-...
...,...,...,...,...,...
1104,L.S.U. and Oklahoma Both Seek to End Title Gam...,2019-12-27T16:25:58-05:00,ATLANTA — The right hand of Lee Morris’s fathe...,New York Times,https://www.nytimes.com/2019/12/27/sports/ncaa...
1105,Can They Kick It? Yes They Can. Ohio State and...,2019-12-27T15:36:57-05:00,"SCOTTSDALE, Ariz. — Pity the plight of college...",New York Times,https://www.nytimes.com/2019/12/27/sports/ncaa...
1106,Drawing the Guantánamo Bay War Court,2019-12-27T18:26:08-05:00,Times Insider explains who we are and what we ...,New York Times,https://www.nytimes.com/2019/12/27/reader-cent...
1107,"Quotation of the Day: Jerry Herman, 88, Compos...",2019-12-27T23:34:59-05:00,“There are only a couple of us who care about ...,New York Times,https://www.nytimes.com/2019/12/27/todayspaper...


In [71]:
problems

[97,
 262,
 262,
 286,
 287,
 287,
 294,
 420,
 497,
 573,
 573,
 620,
 664,
 695,
 742,
 770,
 810,
 853,
 853,
 897,
 935,
 1006,
 1008,
 1008,
 1076]

In [69]:
nyt.to_csv('nyt_df_12.csv', sep='\t')

In [86]:
# find the abnormal content rows in the df:
import numpy as np

# r, = np.where(nyt['content'].isna())
# print(r.tolist(), 'has NaN values in content')

r, = np.where(nyt['content'].str.contains("{") == True)
print(r.tolist(), 'has abnormal values in content')
re_crawl = r.tolist()  # these links need to recrawl by the new method

[3, 4, 5, 8, 12, 17, 39, 54, 62, 73, 98, 122, 125, 132, 138, 192, 204, 216, 217, 237, 238, 270, 291, 294, 337, 338, 345, 349, 351, 382, 414, 415, 418, 424, 425, 432, 458, 465, 467, 504, 505, 538, 568, 569, 574, 577, 594, 612, 617, 628, 633, 656, 673, 692, 697, 700, 703, 718, 727, 728, 730, 737, 743, 747, 770, 774, 807, 815, 822, 841, 846, 855, 869, 899, 921, 930, 933, 978, 994, 1007, 1025, 1040, 1076, 1084] has abnormal values in content


In [74]:
# get some the urls for the problematic ones:

print(full_links[4])
print(full_links[97])
print(full_links[262])
print(full_links[286])

https://www.nytimes.com/2019/11/30/arts/latin-dictionary.html
https://www.nytimes.com/interactive/2019/12/02/us/opioid-crisis-high-school-teenagers.html
https://www.nytimes.com/interactive/2019/12/02/nyregion/nyc-subway-map.html
https://www.nytimes.com/interactive/2019/12/07/us/video-games-child-sex-abuse.html


### Fixing those with code snippets inside content

In [87]:
# find('section').get_text() contains some code in the text
# now we recrawl the articles of these urls from re_crawl list by the following algorithm
header = {'User-Agent' : 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'}#，'Referer' : 'http://www.nytimes.com/'}

for num in re_crawl:
    response = requests.get(full_links[num], header)
    soup = BeautifulSoup(response.text,'html.parser')
    content = soup.find('body').find('section', {'name':'articleBody'}).find_all('p')
    pgs = []
    for pg in content:
        pgs.append(pg.get_text().strip())
    text = '\n'.join(pgs)
    #print(text)
    
    nyt.loc[num, 'content'] = text
    
display(nyt)

,title,timestamp,content,media,link
0,Trump’s Intervention in SEALs Case Tests Penta...,2019-11-30T20:47:50-05:00,"He was limp and dusty from an explosion, consc...",New York Times,https://www.nytimes.com/2019/11/30/us/politics...
1,Prime Mover: How Amazon Wove Itself Into the L...,2019-11-30T06:00:18-05:00,Amazon EverywherePrime Mover: How Amazon Wove ...,New York Times,https://www.nytimes.com/2019/11/30/business/am...
2,Students Fainting From Hunger in Venezuela’s F...,2019-11-30T05:00:23-05:00,"BOCA DE UCHIRE, Venezuela — Hundreds of childr...",New York Times,https://www.nytimes.com/2019/11/30/world/ameri...
3,2020 Democratic Candidates Wage Escalating Fig...,2019-11-30T05:00:15-05:00,"WALPOLE, N.H. — Pete Buttigieg has a nifty pol...",New York Times,https://www.nytimes.com/2019/11/30/us/politics...
4,Latin Dictionary’s Journey: A to Zythum in 125...,2019-11-30T12:34:51-05:00,MUNICH — When German researchers began working...,New York Times,https://www.nytimes.com/2019/11/30/arts/latin-...
...,...,...,...,...,...
1104,L.S.U. and Oklahoma Both Seek to End Title Gam...,2019-12-27T16:25:58-05:00,ATLANTA — The right hand of Lee Morris’s fathe...,New York Times,https://www.nytimes.com/2019/12/27/sports/ncaa...
1105,Can They Kick It? Yes They Can. Ohio State and...,2019-12-27T15:36:57-05:00,"SCOTTSDALE, Ariz. — Pity the plight of college...",New York Times,https://www.nytimes.com/2019/12/27/sports/ncaa...
1106,Drawing the Guantánamo Bay War Court,2019-12-27T18:26:08-05:00,Times Insider explains who we are and what we ...,New York Times,https://www.nytimes.com/2019/12/27/reader-cent...
1107,"Quotation of the Day: Jerry Herman, 88, Compos...",2019-12-27T23:34:59-05:00,“There are only a couple of us who care about ...,New York Times,https://www.nytimes.com/2019/12/27/todayspaper...


In [88]:
nyt.to_csv('nyt_df_12.csv', sep='\t')

### Fixing those special articles with no 'section' part

In [60]:
i = 97
response = requests.get(full_links[i], header)
soup = BeautifulSoup(response.text,'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html xmlns:og="http://opengraphprotocol.org/schema/">
 <head>
  <title data-rh="true">
   The Class of 2000 ‘Could Have Been Anything.’ Until Opioids Hit. - The New York Times
  </title>
  <meta content="en-US" data-rh="true" itemprop="inLanguage"/>
  <meta content="2019-12-02T10:00:04.000Z" data-rh="true" itemprop="datePublished dateCreated" property="article:published"/>
  <meta content="2019-12-09T03:53:28.976Z" data-rh="true" itemprop="dateModified" property="article:modified"/>
  <meta content="en" data-rh="true" http-equiv="Content-Language"/>
  <meta content="noarchive" data-rh="true" name="robots"/>
  <meta content="100000006818290" data-rh="true" itemprop="identifier" name="articleid"/>
  <meta content="nyt://interactive/3c543abf-5ffd-57cf-9f31-f1e83948f96a" data-rh="true" itemprop="identifier" name="nyt_uri"/>
  <meta content="pubp://event/d3b21657522542dbbb48cd49f6c49fe3" data-rh="true" itemprop="identifier" name="pubp_event_id"/>
  <meta content="The high s

In [59]:
# an example test
entry = []
i = 97
response = requests.get(full_links[i], header)
soup = BeautifulSoup(response.text,'html.parser')

#title
title = soup.find('h1').get_text().strip()
# print(title)
entry.append(title)

# timestamp
timestamp = soup.find('time')['datetime']
# print(timestamp)
if timestamp != []:
    entry.append(timestamp.strip())  # if there is 'Last Updated', delete the latter part
else:
    entry.append(None)
    print("timestamp for "+str(i)+" not found")
    problems.append(i)

# content
content = soup.find('body').find('section', {'name':'articleBody'}).get_text().strip()
#print(content)
if content == '' or content is None:
    entry.append(None)
    print("content for "+str(i)+" not found")
    problems.append(i)
else:  # content is not None:
    entry.append(content)


# media
entry.append('New York Times')

# link
entry.append(full_links[i])

#print(entry)

nyt.loc[i] = entry

nyt.loc[0,'content']

AttributeError: 'NoneType' object has no attribute 'get_text'

In [94]:
# further test solutions

num = 97
response = requests.get(full_links[num], header)
soup = BeautifulSoup(response.text,'html.parser')
content = soup.find('body').find_all('p')
pgs = []
for pg in content:
    pgs.append(pg.get_text().strip())
text = '\n'.join(pgs)
print(text)

By Dan LevinDec. 2, 2019
The Minford High School Class of 2000, in rural Minford, Ohio, began its freshman year as a typical class. It had its jocks and its cheerleaders, its slackers and its overachievers.
But by the time the group entered its final year, its members said, painkillers were nearly ubiquitous, found in classrooms, school bathrooms and at weekend parties.
Over the next decade, Scioto County, which includes Minford, would become ground zero in the state’s fight against opioids. It would lead Ohio with its rates of fatal drug overdoses, drug-related incarcerations and babies born with neonatal abstinence syndrome.
To understand both the scope and the devastating consequences of what is now a public health crisis, we talked to dozens of members of the Class of 2000. Many opened up to us about struggles with addiction, whether their own or their relatives’. They told us about the years lost to getting high and in cycling in and out of jail, prison and rehab. They mourned the

In [96]:
# fix the content part (leave the timestamp for now)

problems = list(dict.fromkeys(problems))
print(problems)

header = {'User-Agent' : 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'}#，'Referer' : 'http://www.nytimes.com/'}

for num in problems:
    response = requests.get(full_links[num], header)
    soup = BeautifulSoup(response.text,'html.parser')
    content = soup.find('body').find_all('p')
    pgs = []
    for pg in content:
        pgs.append(pg.get_text().strip())
    text = '\n'.join(pgs)
    #print('****************************** The content for '+str(num))
    #print(text)
    
    nyt.loc[num, 'content'] = text
    
display(nyt)

[97, 262, 286, 287, 294, 420, 497, 573, 620, 664, 695, 742, 770, 810, 853, 897, 935, 1006, 1008, 1076]


,title,timestamp,content,media,link
0,Trump’s Intervention in SEALs Case Tests Penta...,2019-11-30T20:47:50-05:00,"He was limp and dusty from an explosion, consc...",New York Times,https://www.nytimes.com/2019/11/30/us/politics...
1,Prime Mover: How Amazon Wove Itself Into the L...,2019-11-30T06:00:18-05:00,Amazon EverywherePrime Mover: How Amazon Wove ...,New York Times,https://www.nytimes.com/2019/11/30/business/am...
2,Students Fainting From Hunger in Venezuela’s F...,2019-11-30T05:00:23-05:00,"BOCA DE UCHIRE, Venezuela — Hundreds of childr...",New York Times,https://www.nytimes.com/2019/11/30/world/ameri...
3,2020 Democratic Candidates Wage Escalating Fig...,2019-11-30T05:00:15-05:00,"WALPOLE, N.H. — Pete Buttigieg has a nifty pol...",New York Times,https://www.nytimes.com/2019/11/30/us/politics...
4,Latin Dictionary’s Journey: A to Zythum in 125...,2019-11-30T12:34:51-05:00,MUNICH — When German researchers began working...,New York Times,https://www.nytimes.com/2019/11/30/arts/latin-...
...,...,...,...,...,...
1104,L.S.U. and Oklahoma Both Seek to End Title Gam...,2019-12-27T16:25:58-05:00,ATLANTA — The right hand of Lee Morris’s fathe...,New York Times,https://www.nytimes.com/2019/12/27/sports/ncaa...
1105,Can They Kick It? Yes They Can. Ohio State and...,2019-12-27T15:36:57-05:00,"SCOTTSDALE, Ariz. — Pity the plight of college...",New York Times,https://www.nytimes.com/2019/12/27/sports/ncaa...
1106,Drawing the Guantánamo Bay War Court,2019-12-27T18:26:08-05:00,Times Insider explains who we are and what we ...,New York Times,https://www.nytimes.com/2019/12/27/reader-cent...
1107,"Quotation of the Day: Jerry Herman, 88, Compos...",2019-12-27T23:34:59-05:00,“There are only a couple of us who care about ...,New York Times,https://www.nytimes.com/2019/12/27/todayspaper...


In [97]:
# test if there are none in content any more
nyt['content'].isnull().values.any()

False

In [98]:
nyt.to_csv('nyt_df_12.csv', sep='\t')

## Previous Testings

In [12]:

url = 'https://www.nytimes.com/2019/12/27/us/politics/trumps-human-rights-congress.html'
header = {'User-Agent' : 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'}#，'Referer' : 'http://www.chinadaily.com.cn/'}
response = requests.get(url, header)
soup = BeautifulSoup(response.text,'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html class="story" lang="en-US" xmlns:og="http://opengraphprotocol.org/schema/">
 <head>
  <title data-rh="true">
   Congress Wants to Force Trump’s Hand on Human Rights in China and Beyond - The New York Times
  </title>
  <meta content="en-US" data-rh="true" itemprop="inLanguage"/>
  <meta content="2019-12-27T18:30:11.000Z" data-rh="true" itemprop="datePublished dateCreated" property="article:published"/>
  <meta content="2019-12-27T20:54:16.000Z" data-rh="true" itemprop="dateModified" property="article:modified"/>
  <meta content="en" data-rh="true" http-equiv="Content-Language"/>
  <meta content="noarchive" data-rh="true" name="robots"/>
  <meta content="100000006892038" data-rh="true" itemprop="identifier" name="articleid"/>
  <meta content="nyt://article/061e4b0a-1e56-5d16-afb0-ded26818ce03" data-rh="true" itemprop="identifier" name="nyt_uri"/>
  <meta content="pubp://event/9611cd3d638748a6b1ddf0992b66a4b8" data-rh="true" itemprop="identifier" name="pubp_event_id

In [14]:
content = soup.find('body').find('section', {'name':'articleBody'}).get_text()
content

'WASHINGTON — In a rare show of bipartisan unity, Republicans and Democrats are planning to try to force President Trump to take a more active stand on human rights in China, preparing veto-proof legislation that would punish top Chinese officials for detaining more than one million Muslims in internment camps. The effort comes amid growing congressional frustration with Mr. Trump’s unwillingness to challenge China over human rights abuses, despite vivid news reports this year outlining atrocities, or to confront such issues globally. To press Mr. Trump into action on China, lawmakers plan to move ahead with legislation that would punish Beijing for its repression of ethnic Uighur Muslims, with enough supporters to compel the president to sign or risk being overruled by Congress ahead of the 2020 election. A version of the legislation, known as the Uighur Human Rights Policy Act, passed both the House and Senate this year, but its path to the White House was stalled this month by a pro

In [19]:
title = soup.find('h1').get_text().strip()
print(title)

timestamp = soup.find('time')['datetime']
print(timestamp)

Congress Wants to Force Trump’s Hand on Human Rights in China and Beyond
2019-12-27T13:30:11-05:00
